# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt
import requests
import numpy as np




# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port elizabeth,-33.9180,25.5701,17.40,89,75,3.60,ZA,1697490926
1,1,ergani,38.2653,39.7621,16.21,64,15,0.69,TR,1697491563
2,2,coquimbo,-29.9533,-71.3436,16.85,67,0,6.17,CL,1697491349
3,3,bethel,41.3712,-73.4140,16.40,65,75,3.60,US,1697491202
4,4,portland,45.5234,-122.6762,19.79,89,75,3.60,US,1697490947


In [4]:
countdf = city_data_df.count()
countdf

City_ID       557
City          557
Lat           557
Lng           557
Max Temp      557
Humidity      557
Cloudiness    557
Wind Speed    557
Country       552
Date          557
dtype: int64

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

humidity_point_size = city_data_df["Humidity"].astype
# Configure the map plot
city_humidity_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "humidity_point_size",
    color = "City",
    alpha = .9,
)

# Display the map
city_humidity_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_cities = city_data_df.loc[(city_data_df["Max Temp"] > 19) & (city_data_df["Humidity"] < 55)]
city_data_df.dropna()
narrow_cities


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
17,17,los angeles,34.0522,-118.2437,36.43,50,0,4.12,US,1697491007
19,19,bardai,11.6500,7.2333,25.61,36,99,2.09,NG,1697491565
23,23,carnarvon,-24.8667,113.6333,23.20,54,0,5.40,AU,1697491394
33,33,aswan,24.0934,32.9070,24.94,30,0,6.45,EG,1697491567
40,40,san julian,21.0167,-102.1667,25.14,27,20,5.44,MX,1697491568
...,...,...,...,...,...,...,...,...,...,...
536,536,tegueste,28.5167,-16.3167,24.99,42,20,1.79,ES,1697491619
542,542,el gouna,27.3942,33.6782,25.11,46,0,8.25,EG,1697491619
544,544,ciudad bolivar,8.1222,-63.5497,32.72,47,95,1.37,VE,1697491620
546,546,senanga,-16.1167,23.2667,26.69,41,94,1.11,ZM,1697491620


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
copied_columns = ['City', 'Country', 'Lat', 'Lng', 'Humidity']
new_hotel_df = narrow_cities[copied_columns].copy()
new_hotel_df['Hotel Name'] = ''   
new_hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data


,City,Country,Lat,Lng,Humidity,Hotel Name
17,los angeles,US,34.0522,-118.2437,50,
19,bardai,NG,11.6500,7.2333,36,
23,carnarvon,AU,-24.8667,113.6333,54,
33,aswan,EG,24.0934,32.9070,30,
40,san julian,MX,21.0167,-102.1667,27,
...,...,...,...,...,...,...
536,tegueste,ES,28.5167,-16.3167,42,
542,el gouna,EG,27.3942,33.6782,46,
544,ciudad bolivar,VE,8.1222,-63.5497,47,
546,senanga,ZM,-16.1167,23.2667,41,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in new_hotel_df.iterrows():
    Lat = row["Lat"]
    Lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = [Lat, Lng]
    params["bias"] = 0.1
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        new_hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{new_hotel_df.loc[index, 'City']} - nearest hotel: {new_hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
new_hotel_df

Starting hotel search
los angeles - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
aswan - nearest hotel: No hotel found
san julian - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
calipatria - nearest hotel: No hotel found
hobbs - nearest hotel: No hotel found
orchard homes - nearest hotel: No hotel found
las lomitas - nearest hotel: No hotel found
wickenburg - nearest hotel: No hotel found
opuwo - nearest hotel: No hotel found
paulden - nearest hotel: No hotel found
guerrero negro - nearest hotel: No hotel found
biltine - nearest hotel: No hotel found
coronel marcelino mariduena - nearest hotel: No hotel found
mount isa - nearest hotel: No hotel found
fort scott - nearest hotel: No hotel found
awjilah - nearest hotel: No hotel found
tataouine - nearest hotel: No hotel found
sabha - nearest hotel: No hotel found
masjed soleyman - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel fou

,City,Country,Lat,Lng,Humidity,Hotel Name
17,los angeles,US,34.0522,-118.2437,50,No hotel found
19,bardai,NG,11.6500,7.2333,36,No hotel found
23,carnarvon,AU,-24.8667,113.6333,54,No hotel found
33,aswan,EG,24.0934,32.9070,30,No hotel found
40,san julian,MX,21.0167,-102.1667,27,No hotel found
...,...,...,...,...,...,...
536,tegueste,ES,28.5167,-16.3167,42,No hotel found
542,el gouna,EG,27.3942,33.6782,46,No hotel found
544,ciudad bolivar,VE,8.1222,-63.5497,47,No hotel found
546,senanga,ZM,-16.1167,23.2667,41,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
city_humidity_plot_2 = new_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "humidity_size",
    color = "City",
    alpha = .6,
)

# Display the map
city_humidity_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)